# Plotting Redbox

A place to draw the root graph as it exists in code.

In [ ]:
from redbox.app import Redbox
from pathlib import Path
from dotenv import load_dotenv
_ = load_dotenv(Path.cwd().parent / ".env")

In [ ]:
app = Redbox()

In [ ]:
from IPython.display import Image, display
from langchain_core.runnables.graph import MermaidDrawMethod

display(
    Image(
        app.graph.get_graph(xray=True).draw_mermaid_png(
            draw_method=MermaidDrawMethod.API,
        )
    )
)